In [4]:
# Import dependencies & setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

import json
import gmaps


# Import API key
from API_Key_Holder import Z_API_Key

In [2]:
# note that the issue we have with the excel file is that we need to "weight" the median home values
# across different areas (so that the weighted areas correspond with Andy's crime data).
# I've used the following website:
# https://www.zillow.com/research/local-market-reports/
# which shows that for Raleigh ~1.4% of total homes in the market are listed for sale. 
# I've made the assumption that all areas/regions/counties within Raleigh all have exactly 1.4% of total homes 
# listed as for sale.
# Therefore, using the data for the number of homes listed for sale in each region I then divided these 
# numbers by 1.4% to get to the total homes in each area. 
# I then weighted the median home values for each segment of Raleigh by the number of homes in each area
# of each segment.

In [5]:
# Sources:
fig = gmaps.figure()
# fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
heat_layer = gmaps.heatmap_layer()